In [ ]:
# load the packages needed --> For Jupyter no installation needed
library(data.table)
library(rvest)
library(magrittr)
library(stringr)

In [ ]:
# The webpage for scraping the results
url <- "https://www.weltfussball.de/alle_spiele/bundesliga-2018-2019/"
# read the whole html-text into a variable
content <- read_html(url)

In [ ]:
################## APPROACH 1 ############################

In [ ]:
# Extract the first column of the table which holds the date
Spieldatum <- content %>%
  html_nodes(".portfolio .standard_tabelle") %>%
  html_nodes("td:nth-of-type(1)") %>% # get first column of table
  html_text()

Spieldatum %>% head()

In [ ]:
# Extract the third column of the table which holds the name of the home team
Heimteam <- content %>%
  html_nodes(".portfolio .standard_tabelle") %>%
  html_nodes("td:nth-of-type(3)") %>% # get third column of table
  html_text()

Heimteam %>% head()

In [ ]:
# Names of teams must not contain any spaces --> Werder Bremen should be WerderBremen
Heimteam <- Heimteam %>% str_replace_all(" ", "") 

Heimteam %>% head()

In [ ]:
# Extract the fifth column of the table which holds the name of the home team
Austeam <- content %>%
  html_nodes(".portfolio .standard_tabelle") %>%
  html_nodes("td:nth-of-type(5)") %>% # get fifth column of table
  html_text()

Austeam %>% head()

In [ ]:
# Names of teams must not contain any spaces --> Werder Bremen should be WerderBremen
Austeam <- Austeam %>% str_replace_all(" ", "")

Austeam %>% head()

In [ ]:
# Extract the sixth column of the table which holds the result of the game as well as the half-time score
Ergebnis <- content %>%
  html_nodes(".portfolio .standard_tabelle") %>%
  html_nodes("td:nth-of-type(6)") %>% # get sixth column of table
  html_node("a") %>% 
  html_text()

Ergebnis %>% head()

In [ ]:
# Extract from the scores only the final results
Ergebnis <- Ergebnis %>% str_extract("^\\d+:\\d+")

Ergebnis %>% head()

In [ ]:
# combine all in one data.table
resultTable <- data.table(Spieldatum,
                          Heimteam, 
                          Austeam,
                          Ergebnis)

resultTable %>% head()

In [ ]:
# fill the missing dates with the previous ones.
resultTable[, Spieldatum := Spieldatum[1], .(cumsum(Spieldatum != ""))]

resultTable %>% head()

In [ ]:
# split the date into 3 columns; 24.08.2018 should be in three columns: 24 | 08 | 2018
resultTable[, c("Tag", "Monat", "Jahr") := tstrsplit(Spieldatum, ".", fixed = TRUE)]

resultTable %>% head()

In [ ]:
# split the result into 2 columns; 0:5 should be in two columns: 0 | 5
resultTable[, c("Heim", "Aus") := tstrsplit(Ergebnis, ":", fixed = TRUE)]

resultTable %>% head()

In [ ]:
# add the dummy column "Bericht" to make the data readable in Excel
resultTable[, Bericht := "Bericht"]

resultTable %>% head()

In [ ]:
################## APPROACH 2 ############################

In [ ]:
# Scrap the html-table as a whole (all columns at once).
full_table <- content %>% 
  html_nodes(".portfolio .standard_tabelle") %>% 
  html_table()

# We see that the table can also be read in total
# The 1. indicates that we have the first element of a list of tables
# From the structure of the website we know that only one table is available
# with class "standard_tabelle".
head(full_table)

In [ ]:
# Verify the data types of the scraped variable "full_table"
full_table %>% class()      # as expected a list of data.frames
full_table[[1]] %>% class() # the first element is a data.frame 
full_table  %>% str()       # get same info with structure  


In [ ]:
# Because we only have one element in the list of data.frames we take this element 
# and save it as a data.table
full_table <- full_table[[1]] %>% setDT()
full_table %>% str()

In [ ]:
# Delete those columns which are not longer needed
# we have to work with column-indices because all column-names are equal
set(full_table, j = c(2L, 4L, 7L, 8L), value = NULL)
full_table %>% head()

In [ ]:
# rename the columns
setnames(full_table, c("Spieldatum", "Heimteam", "Austeam", "Ergebnis"))
full_table %>% head()